Hi trước khi bắt đầu thì mình xin phép giới thiệu một xíu đây là dự án mà mình đã học lý thuyết qua khóa học free trên udacity và đọc một số bài trên gg,kaggle và zhihu. Nên mình có cóp nhặt về một số ý và làm cái project này. Để không bị coi là viết bằng chatgpt nên mình sẽ có viết tắt một xíu nhưng không nhiều đáng kể.
Mọi sai sót xin hãy liên hệ mình qua email: thy.nhy.da@gmail.com

Mình rất cảm kích luôn ạ. Vì học qua những sai lầm sẽ khiến ta nhớ lâu hơn.

Vậy giờ bắt đầu thui lào!!!
###**A/B Testing Overview**
A/B testing là một phương pháp thử nghiệm so sánh giữa 2 phiên bản A và B.Và được sử dụng nhiều trong việc đánh giá chiến dịch marketing mới đây liệu có thành công hơn trước hay không? hay sự thay đổi mới có làm cho thuật toán tốt hơn trước so với mong đợi không bla bla.
Thường khi có sự thay đổi mới trên website, họ sẽ thường thử nghiệm trước và đánh giá xem liệu có nên triển khai không.
### **Vậy A/B Testing được làm như thế nào?**
Đầu tiên chúng ta sẽ chia thành 2 nhóm:
- A(control group): dùng phiên bản hiện tại
- B(experiment group): dùng phiên bản mới

Sau đó sẽ dùng một số metric phù hợp với mục đích, kiểm định giả thuyết (hypothis testing ),... để so sánh và đánh giá.
##**Experiment Overview**
Udacity muốn tăng tỷ lệ hoàn thành khóa học bằng cách lọc ra những người học không nghiêm túc ngay từ bước đăng ký.

Thí nghiệm thay đổi quy trình "Start Free Trial":
- Hỏi học viên có bao nhiêu thời gian học mỗi tuần.
- Nếu < 5 giờ → cảnh báo → gợi ý học miễn phí.
- Nếu ≥ 5 giờ/tuần → tiếp tục đăng ký như bình thường.

Giả thuyết
Việc đặt kỳ vọng sớm sẽ:

- Giảm số học viên bỏ dở do không đủ thời gian.

- Không ảnh hưởng tiêu cực đến những học viên nghiêm túc.

- Tối ưu hỗ trợ từ giảng viên.

Kỹ thuật:

- Chia nhóm A/B dựa trên cookie (người dùng mới).

- Nếu có đăng ký, sẽ theo dõi bằng user-id (chỉ đăng ký thử 1 lần).

Phần mà mình thấy khá quan trọng đó là **metric choice**
###**Metric choice**###
Chúng ta cần 2 loại metric cho thí nghiệm này: invariate(k đổi, bất biến) và evaluation metrics.
- Invariate metrics dùng để kiểm tra chia nhóm có công bằng không – Sanity Check. Để chọn ra những metric này ta cần cân nhắc xem đâu là chỉ số sẽ không bị thay đổi hay ảnh hưởng đáng kể giữa 2 nhóm control và experiment.
- Evaluation metrics đây là chỉ số được kỳ vọng có sự thay đổi nào đó (tăng hoặc giảm), và nó liên quan đến business goals. Mỗi metric ta sẽ set Dmin - minimum change có ý nghĩa thực tiễn (practically significant). Ví dụ như  nếu Conversion Rate có sự thay đổi tăng 1. 5% cho dù có ý nghĩa thống kê(statistically significant) nhưng dmin= 2%  thì không đáng để triển khai vì không có ý nghĩa thực tiễn cho doanh nghiệp.

####**Invariate Metrics- Sanity Checks**
Ở đây ta cân nhắc đến một số metric có thể sẽ k đổi sau khi có sự cập nhật như:
+ Số lượng cookies ở trang overview (unique daily cookies in Overview Page), Dmin = 3000
+ Số lượt click vào nút "Free Trial" (unique daily cookies who clicked), Dmin= 240
+ Tỷ lệ nhấp chuột (CTP) vào nút "Free Trial" (số lượt nhấp/số người xem (cookies)）, Dmin = 0.01

####**Evaluation Metrics - Performance Indicators**
+ Gross Conversion (tỷ lệ chuyển đổi sang dùng thử), Dmin =0.01
+ Retention (tỷ lệ giữ chân), Dmin =0.01
+ Net Conversion(tỷ lệ chuyển đổi sang trả phí), Dmin = 0.0075

#### **Estimating the baseline values of metrics**
Trước khi thử nghiệm thì chúng ta nên biết những metric này như nào trước khi có sự thay đổi đó được gọi là **Giá trị baseline**

| **Item**               | **Description**                                        | **Estimator** |
|------------------------|--------------------------------------------------------|---------------|
| Number of cookies      | Daily unique cookies to view course overview page      | 40,000        |
| Number of clicks       | Daily unique cookies to click Free Trial button        | 3,200         |
| Number of enrollments  | Free Trial enrollments per day                         | 660           |
| CTP                    | CTP on Free Trial button                               | 0.08          |
| Gross Conversion       | Probability of enrolling, given a click                | 0.20625       |
| Retention              | Probability of payment, given enrollment               | 0.53          |
| Net Conversion         | Probability of payment, given click                    | 0.109313      |



In [4]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm




In [5]:
def Standard_Deviation(p,n):
  return math.sqrt(p*(1-p)/n)

In [6]:
ctr_data = pd.read_csv("/content/drive/MyDrive/A_B_Testing/Data/control_data.csv")
ctr_data.describe()

,Pageviews,Clicks,Enrollments,Payments
count,37.000000,37.000000,23.000000,23.000000
mean,9339.000000,766.972973,164.565217,88.391304
std,740.239563,68.286767,29.977000,20.650202
min,7434.000000,632.000000,110.000000,56.000000
25%,8896.000000,708.000000,146.500000,70.000000
50%,9420.000000,759.000000,162.000000,91.000000
75%,9871.000000,825.000000,175.000000,102.500000
max,10667.000000,909.000000,233.000000,128.000000


Như chúng ta thấy, sẽ có một số record sẽ bị null ở côt enrollment, payment.

Việc thiếu dữ liệu này cũng dễ hiểu vì:

Pageviews và clicks thường được ghi nhận ngay khi người dùng truy cập, nên có thể ghi nhận đủ 39 ngày.

Enrollments và payments lại chỉ được ghi nhận sau khi người dùng đã click và trải qua một giai đoạn dùng thử (free trial), nên chỉ có dữ liệu từ 23 ngày đầu — tức là:

Hệ quả tự nhiên của việc người dùng cần thời gian (ví dụ 14 ngày) để chuyển từ click → enroll → payment.

Vì vậy khi phân tích đến enrollment, payment thì ta sẽ so sánh ta cần lấy những record có đầy đủ dữ liệu

In [7]:
ctr_data.dtypes

,0
Date,object
Pageviews,int64
Clicks,int64
Enrollments,float64
Payments,float64


Vì ta k có sử dụng đến cột date nên ta k cần fomart thành dạng date

In [8]:
exp_data = pd.read_csv("/content/drive/MyDrive/A_B_Testing/Data/experiment_data.csv")
exp_data.describe()

,Pageviews,Clicks,Enrollments,Payments
count,37.000000,37.000000,23.000000,23.000000
mean,9315.135135,765.540541,148.826087,84.565217
std,708.070781,64.578374,33.234227,23.060841
min,7664.000000,642.000000,94.000000,34.000000
25%,8881.000000,722.000000,127.000000,69.000000
50%,9359.000000,770.000000,142.000000,91.000000
75%,9737.000000,827.000000,172.000000,99.000000
max,10551.000000,884.000000,213.000000,123.000000


In [9]:
exp_data.dtypes

,0
Date,object
Pageviews,int64
Clicks,int64
Enrollments,float64
Payments,float64


In [10]:
#chúng ta sẽ thêm baseline vào một dictionary để một tý sử dụng để so sánh nhé
baseline = {"Cookies":40000,"Clicks":3200,"Enrollments":660,"CTP":0.08,"GConversion":0.20625,
           "Retention":0.53,"NConversion":0.109313}

###**Ước lượng độ lệch chuẩn (Standard Deviation) - SD**
Độ lệch chuẩn có liên quan đến sample size và confidence interval

Ví dụ như : độ lệch chuẩn càng lớn thì độ tin cậy của kết quả càng thấp vì thế ta cần mẫu( sample size) lớn hơn

Giả định rằng mỗi ngày có 5,000 cookies truy cập trang giới thiệu khóa học, chúng ta cần ước tính SD cho evaluation metrics. Sample size cần tính nên nhỏ hơn tổng số liệu gốc (population) đã thu thập.
Tiếp theo ta cần scale các metrics
###**Scaling**

In [11]:
baseline['Cookies'] = 5000
baseline['Clicks']=baseline['Clicks']*(5000/40000)
baseline['Enrollments']=baseline['Enrollments']*(5000/40000)
baseline


{'Cookies': 5000,
 'Clicks': 400.0,
 'Enrollments': 82.5,
 'CTP': 0.08,
 'GConversion': 0.20625,
 'Retention': 0.53,
 'NConversion': 0.109313}

###**Estimating Analytically**
Chúng ta sẽ ước lượng SD bằng công thức toán học thay vì mô phỏng thực nghiệm (bootstrap), thì đó gọi là estimating analytically.

Đối với các metric là xác suất thì phân phối của chúng là phân phối nhị thức (biomially distributed), vì thế ta có thể sử dụng cthuc tính SD dưới đây:

### Công thức tính độ lệch chuẩn (Standard Deviation - SD):

$$
\text{SD} = \sqrt{\frac{p(1 - p)}{n}}
$$

Trong đó:

- `p` là tỷ lệ (ví dụ: tỷ lệ chuyển đổi baseline)
- `n` là số mẫu (ví dụ: số người click)

Giả định phương sai phân phối nhị phân (binomial variance) chỉ hợp lệ khi:

“Unit of Diversion” (đơn vị chia nhóm) bằng với

“Unit of Analysis” (đơn vị dùng để tính toán metric – mẫu số trong công thức chỉ số)

**Gross Conversion**
= Số người dùng đăng ký dùng thử / Số cookie đã nhấp vào nút dùng thử.
Hay nói cách khác, đây là xác suất một người đăng ký sau khi đã nhấp chuột.

In [12]:
#Bây giờ ta sẽ tính SD của GC trước nhé
GC = {}
GC['d_min'] = 0.01
GC['p'] = baseline['Enrollments']/baseline['Clicks']
GC['n'] = baseline['Clicks']
GC['sd'] = round(Standard_Deviation(GC['p'],GC['n']),4)
GC

{'d_min': 0.01, 'p': 0.20625, 'n': 400.0, 'sd': 0.0202}

**Retention** = No. Paying users(enrolled after 14 days)/ No. total enrrolled users.Tức là = số người trả tiền / số người đã đăng ký (enrolled)

Nhưng có một điều lưu ý rằng the sample size là số user đăng ký. Trong trường hợp này, unit of diversion và unit of analysis không bằng nhau nên việc ước lượng phương sai (variance) bằng công thức phân tích thông thường sẽ không đủ chính xác.

Nếu có dữ liệu thực tế đầy đủ, ta nên ước lượng phương sai bằng phương pháp thực nghiệm (empirically).

In [13]:
R= {}
R['d_min'] = 0.01
R['p'] = baseline['Retention']
R['n'] = baseline['Enrollments']
R['sd'] = round(Standard_Deviation(R['p'],R['n']),4)
R['sd']

0.0549

Net Conversion =
Số người đã thanh toán / Số cookie đã nhấp vào nút dùng thử

→ Tức là: Xác suất một người sẽ thanh toán sau khi đã nhấp chuột vào nút dùng thử.

Lúc này sample size là số cookie đã nhấp vào nút Free Trial. Do đó Unit of analysis và unit of diversion có giống nhau. Nên kết quả ước lượng sẽ khá chính xác.


In [14]:
# Tính Net Conversion (NC)
NC = {}
NC['d_min'] = 0.0075
NC['p'] = baseline['NConversion']
NC['n'] = baseline['Clicks']
NC['sd'] = round(Standard_Deviation(NC['p'],NC['n']),4)
NC['sd']

0.0156

###**Experiment Sizing**
Phần này chúng ta cần tính số lượng mẫu tối thiểu n cần cho mỗi nhóm để kiểm tra giả thuyết rằng có một sự khác biệt có ý nghĩa thực tiễn (d, tức là Dmin) giữa hai nhóm.

$$
n = \frac{\left(Z_{1 - \frac{\alpha}{2}} \cdot sd_1 + Z_{1 - \beta} \cdot sd_2\right)^2}{d^2}
$$

**with:**

$$
sd_1 = \sqrt{p(1 - p) + p(1 - p)}
$$

$$
sd_2 = \sqrt{p(1 - p) + (p + d)\left(1 - (p + d)\right)}
$$

trong đó:
- sd 1:Độ lệch chuẩn với baseline (giá trị ban đầu)
- sd 2: Độ lệch chuẩn với tỷ lệ sau thay đổi (có d)
###**Tính z-score và standard deviations**

In [15]:
def get_sds(p,d):
  sd1 = math.sqrt(2*p*(1-p))
  sd2 = math.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
  sds = [sd1,sd2]
  return sds

def get_z_score(alpha):
  return norm.ppf(alpha)

#INPUT: sd1 cho baseline, sd2 cho su thay doi ky vong, alpha, beta,d-d_min, p-baseline estimate p
def get_sampSize(sds,alpha,beta,d):
  n = pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
  return n


Bây giờ ta sẽ tính sample size cho từng metric và sẽ chọn ra sample size cao nhất.

In [16]:
print(GC)
print(R)
print(NC)

{'d_min': 0.01, 'p': 0.20625, 'n': 400.0, 'sd': 0.0202}
{'d_min': 0.01, 'p': 0.53, 'n': 82.5, 'sd': 0.0549}
{'d_min': 0.0075, 'p': 0.109313, 'n': 400.0, 'sd': 0.0156}


In [17]:
#Gross Conversion
GC['SampSize'] = round(get_sampSize(get_sds(GC['p'],GC['d_min']),0.05,0.2,GC['d_min']))
GC['SampSize']

25835

Có nghĩa rằng ta cần ít nhất 25,835 cookies có click vào nút free trial của mỗi nhóm. Dựa trên CTP của baseline =0.08

--> pageviews = 25835/0.08 =322,938 cho mỗi nhóm

vậy tổng là 645,875 pageviews

In [18]:
GC['SampSize'] = round(GC['SampSize']/0.08*2)
GC['SampSize']

645875

Retention

In [19]:
R['SampSize'] = round(get_sampSize(get_sds(R['p'],R['d_min']),0.05,0.2,R['d_min']))
R['SampSize']

39087

In [20]:
R['SampSize'] = (R['SampSize']/GC['p'])*1/0.08*2
R['SampSize']

4737818.181818183

Cái metric này cần đến 4 triệu pageview, điều này thật là cần rất nhiều thời gian, bởi vì chúng ta chỉ có thể thu thập 40,000 pageview mỗi ngày. Điều đó mất hơn 100 ngày. Nên chúng ta sẽ loại bỏ metric này vì nếu dùng kết quả này sẽ bị bias

Net Conversion

In [21]:
NC['SampSize'] = round(get_sampSize(get_sds(NC['p'],NC['d_min']),0.05,0.2,NC['d_min']))
NC['SampSize']

27413

In [22]:
NC['SampSize'] = NC['SampSize']/0.08*2
NC['SampSize']

685325.0

Đối với Net Conversion ta cần đến 685,325 pageview. Vì nó cao hơn GC nên ta sẽ chọn số pageview này

##**Analyzing collected Data**
###**Sanity Checks**
Ta cần thực hiện Sanity check để đảm bảo k có sai lệch hay lỗi dữ liệu trong quá trình thu thập dữ liệu.

Chúng ta có 3 invariant metric:
- Số cookie xem trang giới thiệu khóa học (Number of Cookies in Course Overview Page)

- Số người click vào nút Free Trial (Number of Clicks on Free Trial Button)

- Tỷ lệ Click-Through Probability (CTP) – xác suất nhấp = số click / số cookie xem trang

Đầu tiên, Ta tiến hành sanity check với pageview

In [23]:
pageviews_cont=ctr_data['Pageviews'].sum()
pageviews_exp=exp_data['Pageviews'].sum()
pageviews_total=pageviews_cont+pageviews_exp
print ("number of pageviews in control:", pageviews_cont)
print ("number of Pageviewsin experiment:" ,pageviews_exp)

number of pageviews in control: 345543
number of Pageviewsin experiment: 344660


Chúng ta kỳ vọng rằng khoảng 50% trong tổng số người dùng sẽ vào nhóm control, tức là xác suất một người được chia vào nhóm control là 0.5. Khi đó tổng số người vào nhóm control là một biến **ngẫu nhiên nhị thức (Binomial)**

Vì dữ liệu cũng lớn đủ để chúng ta sử dụng định lý giới hạn trung tâm (Central Limit Theorem) cho phân phối Binomial

$$
X \sim N\left(p, \sqrt{\frac{p(1 - p)}{N}} \right)
$$
Điều chúng ta muốn kiểm tra là liệu tỷ lệ quan sát được p_hat( tức là số mẫu trong nhóm control chia cho tổng số mẫu trong cả hai nhóm)
có khác biệt đáng kể so với p=0.5 hay không.

Để làm được điều đó, chúng ta có thể tính khoảng sai số cho phép (margin of error) ở mức độ tin cậy 95%
$$
ME = Z_{1 - \frac{\alpha}{2}} \cdot SD
$$
Và cuối cùng chúng ta sẽ xác định được khoảng tin cậy (Confidence interval)
$$
CI = [\hat{p} - ME,\ \hat{p} + ME]
$$
Khi mà p nằm trong khoảng này thì việc kiểm tra này được thông qua.

In [24]:
p=0.5
alpha = 0.05
p_hat = round(pageviews_cont/(pageviews_total),4)
sd=round(math.sqrt(p*(1-p)/(pageviews_total)),4)
ME=round(get_z_score(1-(alpha/2))*sd,4)
print("Confidence interval nằm ở khoảng từ",p-ME,"đến",p+ME,  "\nVậy",p_hat, "có trong khoảng này?")


Confidence interval nằm ở khoảng từ 0.4988 đến 0.5012 
Vậy 0.5006 có trong khoảng này?


p_hat quan sát được từ dữ liệu nằm trong khoảng tin cậy Điều này cho thấy rằng sự khác biệt về số lượng người giữa 2 nhóm (control và experiment) không quá lớn và có thể chấp nhận được về mặt thống kê.

Tiếp theo ta check metric số cookies click vào nút free trial

In [25]:
clicks_contr = ctr_data['Clicks'].sum()
clicks_exp = exp_data['Clicks'].sum()
clicks_total = clicks_contr + clicks_exp

p_hat = round(clicks_contr/clicks_total,4)
sd = round(math.sqrt(p*(1-p)/clicks_total),4)
ME = round(get_z_score(1-(alpha/2))*sd,4)
print("Confidence interval nằm ở khoảng từ",p-ME,"đến",p+ME,  "\nVậy",p_hat, "có trong khoảng này?")

Confidence interval nằm ở khoảng từ 0.4959 đến 0.5041 
Vậy 0.5005 có trong khoảng này?


Quá tốt rùi, metric này cũng đã được pass. Tiếp theo là metric cuối cùng CTP

CTP là tỷ lệ lượt click trên số pageview

Trong trường hợp này, chúng ta muốn đảm bảo rằng tỷ lệ nhấp chuột trên mỗi lượt xem trang (tức CTP quan sát được) là xấp xỉ nhau giữa hai nhóm (control và experiment), bởi vì đây là chỉ số không được kỳ vọng thay đổi do thí nghiệm.

Để kiểm tra điều này, ta sẽ:

- Tính CTP trong mỗi nhóm (control và experiment),

- Sau đó tính khoảng tin cậy (confidence interval) cho sự khác biệt kỳ vọng giữa hai nhóm này.

Nói cách khác:
Chúng ta kỳ vọng không có sự khác biệt giữa hai nhóm, tức là:

CTPexp - CTPcont = 0

→ Và nếu có sai số, thì sai số đó phải nằm trong khoảng tin cậy cho phép, được tính toán dựa trên độ lệch chuẩn (standard error).

$SD_{pool} = \sqrt{\hat{p}_{pool}(1 - \hat{p}_{pool})\left( \frac{1}{N_{cont}} + \frac{1}{N_{exp}} \right)}$

$\hat{p}_{pool} = \frac{x_{cont} + x_{exp}}{N_{cont} + N_{exp}}$



In [26]:
ctp_cont = clicks_contr/pageviews_cont
ctp_exp = clicks_exp/pageviews_exp
d_hat = round(ctp_exp-ctp_cont,4)
p_pooled = round((clicks_total)/(pageviews_total),4)
sd_pooled = math.sqrt(p_pooled*(1-p_pooled)*(1/pageviews_cont+1/pageviews_exp))
ME = round(get_z_score(1-(alpha/2))*sd_pooled,4)
print("Confidence interval nằm ở khoảng từ",0-ME,"đến",0+ME, "\nVậy",d_hat, "có trong khoảng này?")

Confidence interval nằm ở khoảng từ -0.0013 đến 0.0013 
Vậy 0.0001 có trong khoảng này?


thật tuyệt zời. Có vẻ như việc kiểm tra metric này cũng đã vượt qua một cách xuất sắc rùi.

Bước tiếp theo là so sánh sự thay đổi giữa nhóm thử nghiệm (experiment) và nhóm đối chứng (control) trên các metric đánh giá (ví dụ: Gross Conversion, Net Conversion...).
####**Gross Conversion**
Ý nghĩa thống kê (Statistical significance): Nếu khoảng tin cậy (CI) không chứa số 0, ta có thể kết luận rằng có sự thay đổi thực sự giữa hai nhóm.

 Vì sao lại là số 0?
Vì 0 thể hiện "không có khác biệt" giữa hai nhóm – nên nếu CI không bao gồm 0, ta tự tin có sự thay đổi.

Ý nghĩa thực tiễn (Practical significance): Nếu khoảng tin cậy không chứa mốc giá trị ý nghĩa thực tế (ví dụ: ±0.01 hoặc ±0.02 tùy theo bài toán), thì sự thay đổi là đủ lớn để có giá trị với doanh nghiệp.

 Ví dụ: Nếu sự cải thiện chỉ là 0.001, dù có ý nghĩa thống kê, nhưng nó quá nhỏ để quan trọng về mặt kinh doanh.

Khi EDA ta thấy dữ liệu:

Pageviews và Clicks: trong 39 ngày

Enrollments và Payments: chỉ có 23 ngày

 Khi ta phân tích các metric liên quan đến Enrollments hoặc Payments, ta chỉ được dùng dữ liệu pageviews và clicks tương ứng trong 23 ngày, không dùng toàn bộ 39 ngày – nếu không sẽ bị lệch dữ liệu (gây sai lệch về tỷ lệ).

In [27]:
clicks_cont = ctr_data['Clicks'].loc[ctr_data['Enrollments'].notnull()].sum()
clicks_exp = exp_data['Clicks'].loc[exp_data['Enrollments'].notnull()].sum()


In [28]:
enrollments_cont = ctr_data['Enrollments'].sum()
enrollments_exp = exp_data['Enrollments'].sum()
GC_cont = enrollments_cont / clicks_cont
GC_exp = enrollments_exp / clicks_exp
GC_pooled = (enrollments_cont+enrollments_exp)/(clicks_cont+clicks_exp)
GC_sd_pooled = math.sqrt(GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp))
GC_ME = round(get_z_score(1-alpha/2)*GC_sd_pooled,4)
GC_diff = round(GC_exp-GC_cont,4)
print("Thử nghiệm này làm thay đổi Gross Conversion khoảng",GC_diff*100,"%")
print("Confidence Interval: [",GC_diff-GC_ME,",",GC_diff+GC_ME,"]")
print ("Sự thay đổi được xem là có ý nghĩa thống kê nếu CI k bao gồm 0. Trong TH đó, nó được coi là có ý nghĩa thực tiễn nếu",-GC['d_min'], "cũng k nằm trong khoảng tin cậy")

Thử nghiệm này làm thay đổi Gross Conversion khoảng -2.06 %
Confidence Interval: [ -0.0292 , -0.012 ]
Sự thay đổi được xem là có ý nghĩa thống kê nếu CI k bao gồm 0. Trong TH đó, nó được coi là có ý nghĩa thực tiễn nếu -0.01 cũng k nằm trong khoảng tin cậy


Theo kết quả này, đã có sự thay đổi do thử nghiệm, và sự thay đổi đó có ý nghĩa cả về mặt thống kê lẫn thực tiễn.

-  Chúng ta quan sát thấy sự giảm 2.06%, trong khi mức giảm chấp nhận được là trên 1%.
- Điều này có nghĩa là tỷ lệ Gross Conversion (tỷ lệ người đăng ký dùng thử sau khi nhấp vào nút) ở nhóm thử nghiệm (nhóm được hỏi số giờ học mỗi tuần) đã giảm 2%, và sự thay đổi này là có ý nghĩa.

Nói cách khác: ít người hơn đăng ký dùng thử miễn phí sau khi họ nhìn thấy pop-up hỏi thời gian học.
Và vì điều này là có ý nghĩa thống kê và thực tiễn, nên ta có thể tin rằng sự thay đổi là do thử nghiệm, không phải do ngẫu nhiên.

####**Net Conversion**
Gross và Net khác nhau giữa enrollment và payments. Nên khi gross giảm có thể net cũng giảm theo nhưng quan trọng là có đáng kể hay k


In [29]:
payments_cont = ctr_data['Payments'].sum()
payments_exp = exp_data['Payments'].sum()
NC_cont = payments_cont / clicks_cont
NC_exp = payments_exp / clicks_exp
NC_pooled = (payments_cont+payments_exp)/(clicks_cont+clicks_exp)
NC_pooled_sd = math.sqrt(NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp))
NC_ME = round(get_z_score(1-alpha/2)*NC_pooled_sd,4)
NC_diff = round(NC_exp-NC_cont,4)
print("Thử nghiệm này làm thay đổi Net Conversion khoảng",NC_diff*100,"%")
print("Confidence Interval: [",NC_diff-NC_ME,",",round(NC_diff+NC_ME,4),"]")
print ("Sự thay đổi được xem là có ý nghĩa thống kê nếu CI k bao gồm 0. Trong TH đó, nó được coi là có ý nghĩa thực tiễn nếu",-NC['d_min'], "cũng k nằm trong khoảng tin cậy")

Thử nghiệm này làm thay đổi Net Conversion khoảng -0.49 %
Confidence Interval: [ -0.0116 , 0.0018 ]
Sự thay đổi được xem là có ý nghĩa thống kê nếu CI k bao gồm 0. Trong TH đó, nó được coi là có ý nghĩa thực tiễn nếu -0.0075 cũng k nằm trong khoảng tin cậy


###**Double Check with Sign Tests**
Trong kiểm định dấu (sign test) còn được gọi là kiểm định phi tham số, chúng ta có một góc nhìn khác để phân tích kết quả đã thu được – đó là kiểm tra xem xu hướng thay đổi (tăng hoặc giảm) có rõ ràng trong dữ liệu hằng ngày hay không.

Cách làm như sau:

Ta sẽ tính giá trị của metric (chỉ số) mỗi ngày.

Sau đó, đếm số ngày mà giá trị metric của nhóm thử nghiệm (experiment) thấp hơn nhóm đối chứng (control).

Số ngày như vậy sẽ được coi là số lần thành công (successes) trong một biến ngẫu nhiên nhị phân (binomial variable).

Khi đã có số lần thành công và tổng số ngày, ta có thể xét tỷ lệ ngày thành công (proportion of success days) để đánh giá liệu xu hướng giảm (hoặc tăng) có xảy ra một cách có ý nghĩa thống kê hay không.

Nói cách khác:
 Nếu nhóm thử nghiệm thường xuyên có metric thấp hơn nhóm đối chứng trong đa số ngày, ta có dấu hiệu nhất quán cho thấy thay đổi là đáng tin cậy, ngay cả khi không xét đến độ lớn cụ thể của sự khác biệt mỗi ngày.

In [30]:
data = ctr_data.join(other = exp_data,how='inner',lsuffix = '_cont',rsuffix = '_exp')
data.describe()

,Pageviews_cont,Clicks_cont,Enrollments_cont,Payments_cont,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp
count,37.000000,37.000000,23.000000,23.000000,37.000000,37.000000,23.000000,23.000000
mean,9339.000000,766.972973,164.565217,88.391304,9315.135135,765.540541,148.826087,84.565217
std,740.239563,68.286767,29.977000,20.650202,708.070781,64.578374,33.234227,23.060841
min,7434.000000,632.000000,110.000000,56.000000,7664.000000,642.000000,94.000000,34.000000
25%,8896.000000,708.000000,146.500000,70.000000,8881.000000,722.000000,127.000000,69.000000
50%,9420.000000,759.000000,162.000000,91.000000,9359.000000,770.000000,142.000000,91.000000
75%,9871.000000,825.000000,175.000000,102.500000,9737.000000,827.000000,172.000000,99.000000
max,10667.000000,909.000000,233.000000,128.000000,10551.000000,884.000000,213.000000,123.000000


In [31]:
data = data.loc[data['Enrollments_cont'].notnull()]
data.count()

,0
Date_cont,23
Pageviews_cont,23
Clicks_cont,23
Enrollments_cont,23
Payments_cont,23
Date_exp,23
Pageviews_exp,23
Clicks_exp,23
Enrollments_exp,23
Payments_exp,23


In [32]:
a= data['Enrollments_cont']/data['Clicks_cont']
b= data['Enrollments_exp']/data['Clicks_exp']
data['GC'] = np.where(a<b,1,0)
c= data['Payments_cont']/data['Clicks_cont']
d= data['Payments_exp']/data['Clicks_exp']
data['NC'] = np.where(c<d,1,0)
data.head()

,Date_cont,Pageviews_cont,Clicks_cont,Enrollments_cont,Payments_cont,Date_exp,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp,GC,NC
0,"Sat, Oct 11",7723,687,134.0,70.0,"Sat, Oct 11",7716,686,105.0,34.0,0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,"Sun, Oct 12",9288,785,116.0,91.0,0,1
2,"Mon, Oct 13",10511,909,167.0,95.0,"Mon, Oct 13",10480,884,145.0,79.0,0,0
3,"Tue, Oct 14",9871,836,156.0,105.0,"Tue, Oct 14",9867,827,138.0,92.0,0,0
4,"Wed, Oct 15",10014,837,163.0,64.0,"Wed, Oct 15",9793,832,140.0,94.0,0,1


In [39]:
GC_x = data['GC'][data['GC']==1].count()
NC_x = data['NC'][data['NC']==1].count()
n = data['NC'].count()
print("Số trường hợp success của GC:",GC_x,'\n',
      "Số trường hợp success của NC:",NC_x,'\n',
      "Tổng trường hợp",n)

Số trường hợp success của GC: 4 
 Số trường hợp success của NC: 10 
 Tổng trường hợp 23


Việc chúng ta cần làm sau khi đã đếm số ngày mà nhóm thử nghiệm (experiment) có giá trị metric cao hơn nhóm đối chứng (control), là xem liệu số ngày đó có khả năng xảy ra lại trong một thử nghiệm mới hay không (tức là kiểm tra tính có ý nghĩa thống kê).
Chúng ta giả định rằng xác suất để một ngày như vậy xảy ra là ngẫu nhiên (50% cơ hội xảy ra), và sau đó sử dụng phân phối nhị thức (binomial distribution) với p = 0.5 và số lần thử nghiệm (n là tổng số ngày) để tính xác suất của việc kết quả như vậy xảy ra do ngẫu nhiên.

Vì vậy, theo phân phối nhị thức với p = 0.5 và n = số ngày, ta muốn biết xác suất để có x ngày thành công (tức là metric của nhóm thử nghiệm cao hơn nhóm đối chứng).
Bởi vì đây là kiểm định hai phía (two-tailed test), ta cần nhân đôi xác suất đó. Và khi đã có kết quả, ta gọi đó là giá trị p (p-value) và so sánh nó với mức ý nghĩa α (alpha).

Nếu p-value > α thì kết quả không có ý nghĩa thống kê và ngược lại

$$
P(\text{successes}) = \frac{n!}{x! (n - x)!} \cdot p^x \cdot (1 - p)^{n - x}
$$

In [41]:
#tạo một function để tính xác suất với x là số ngày thành công
def get_prob(x,n):
  p = round(math.factorial(n)/(math.factorial(x)*math.factorial(n-x))*0.5**x*0.5**(n-x),4)
  return p

def get_2side_pvalue(x,n):
  p=0
  for i in range(0,x+1):
    p = p + get_prob(i,n)
  return 2*p



In [45]:
print ("Sự thay đổi của GC có ý nghĩa khi",round(get_2side_pvalue(GC_x,n),4),"nhỏ hơn 0.05")
print ("Sự thay đổi của NC có ý nghĩa khi",round(get_2side_pvalue(NC_x,n),4),"nhỏ hơn 0.05")

Sự thay đổi của GC có ý nghĩa khi 0.0026 nhỏ hơn 0.05
Sự thay đổi của NC có ý nghĩa khi 0.6774 nhỏ hơn 0.05


Kết luận vẫn giống như tính toán độ lớn hiệu ứng (effect size) . Sự thay đổi trong Gross Conversion là có ý nghĩa thống kê, trong khi sự thay đổi trong Net Conversion thì không.

##**Conclusion and Recommendation**
Thí nghiệm này được thiết kế nhằm xác định liệu việc lọc học viên dựa trên cam kết thời gian học tập có cải thiện trải nghiệm học tập tổng thể và năng lực hỗ trợ của các huấn luyện viên đối với những học viên có khả năng hoàn thành khóa học hay không, mà không làm giảm đáng kể số lượng học viên tiếp tục sau giai đoạn dùng thử miễn phí.

Một sự giảm có ý nghĩa thống kê và thực tiễn đã được quan sát thấy ở Gross Conversion, nhưng không có sự khác biệt đáng kể nào ở Net Conversion. Điều này có nghĩa là tỷ lệ người ghi danh (enrollment) giảm xuống, nhưng không đi kèm với việc tăng tỷ lệ những người học đủ 14 ngày để phải trả tiền.

--> Với kết quả này, tôi khuyến nghị KHÔNG nên triển khai thay đổi, mà nên tiếp tục thực hiện những thí nghiệm khác.
##**Follow up Experiment**
**Can thiệp trước ghi danh (Pre-enrollment Intervention)**

Thí nghiệm trước đó đã thử nghiệm một hình thức can thiệp bằng cách hỏi học viên về cam kết thời gian học tập – nhằm lọc ra những người dễ nản. Tuy nhiên, bộ lọc này chỉ tập trung vào thời gian, mà không đề cập đến các lý do khác có thể khiến học viên huỷ sớm. Ví dụ, dù học viên cam kết đủ thời gian nhưng thiếu kỹ năng nền tảng, họ vẫn có thể cảm thấy thất vọng và từ bỏ.

Đề xuất nâng cấp:
- Thêm một danh sách kỹ năng tiền đề (pre-requisite skills) vào pop-up hỏi về thời gian cam kết.

Nếu học viên đáp ứng được cả thời gian và kỹ năng, họ sẽ được đưa tới trang đăng ký dùng thử.

Nếu không đáp ứng, họ sẽ được khuyến khích trải nghiệm bản học miễn phí (free version).

Thí nghiệm này có chi phí thấp, vì tận dụng lại hạ tầng và pipeline dữ liệu hiện tại, chỉ cần thay đổi nội dung biểu mẫu (form).

Một thí nghiệm được coi là thành công nếu:

  Gross Conversion (tỉ lệ đăng ký dùng thử) giảm đáng kể (lọc bớt người không phù hợp), nhưng

  Net Conversion (tỉ lệ chuyển thành trả phí) tăng đáng kể (lọc trúng người phù hợp hơn).

- Tạo bài test đầu vào trước khi học:

  Một cách khác để tăng độ hiệu quả của việc sàng lọc là thiết kế một bài test đầu vào ngắn, nhằm kiểm tra kiến thức nền tảng.
Kết hợp bài test đầu vào cùng checklist thời gian và kỹ năng sẽ tạo ra một bộ lọc toàn diện hơn, giúp chọn đúng người học, đúng khóa học, từ đó tăng tỉ lệ hoàn thành và hài lòng.

**Can thiệp sau ghi danh nhưng trước khi trả phí (Post-enrollment Intervention)**

Có nhiều phương án có thể áp dụng ở giai đoạn này, và chúng có thể triển khai song song với giai đoạn pre-enrollment.

**Đề xuất:**

Khuyến khích học viên tham gia nhóm học tập (team).

Nếu học viên được đưa vào một nhóm học để:

- Thảo luận bài vở,

- Chia sẻ khó khăn,

- Có trách nhiệm cùng nhau học,

Thì khả năng vượt qua giai đoạn ban đầu khó khăn và giữ lại đến ngày thứ 14 sẽ cao hơn.

**Thiết lập thí nghiệm:**

Khi ghi danh, học viên được ngẫu nhiên phân vào:

Nhóm kiểm soát (control): Không tham gia team.

Nhóm thí nghiệm (experiment): Được ghép vào team học.

**Giả thuyết gốc (Null Hypothesis):** Việc tham gia nhóm không làm tăng số người học vượt qua 14 ngày dùng thử một cách có ý nghĩa thống kê.

**Đơn vị phân nhóm (Unit of Diversion):** user-id, vì thay đổi diễn ra sau khi học viên tạo tài khoản.

**Chỉ số bất biến (Invariant Metric):** user-id, vì mong đợi sự phân bổ đều giữa 2 nhóm.

**Chỉ số đánh giá (Evaluation Metric):** Retention (tỷ lệ ở lại sau 14 ngày).

-->Triển khai:
Nếu có sự gia tăng đáng kể (về thống kê và thực tiễn) ở Retention, và tác động đến tài nguyên (ví dụ: duy trì nhóm học không quá tốn kém), thì thí nghiệm sẽ được triển khai chính thức.